In [2]:
!pip install pandas

In [3]:
import os
import csv
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from contextlib import contextmanager

In [4]:
def get_files(method:str):
    path = f'/workspace/accuracy/{method}/'
    file_list = os.listdir(path)
    file_list_csv = [file for file in file_list if file.endswith('.csv')]
    seeds = []
    pattern = f'{method}_sparse_output_([0-9]+).csv'
    for file in file_list_csv:
        result = re.search(pattern, file)
        seeds.append(result.group(1))
    return path, file_list_csv, seeds

In [5]:
def data_sorting(file_str):
    data = pd.read_csv(file_str)
    data_sorted = data.sort_values(by="sparsity").drop_duplicates(subset=['sparsity'], keep = 'first')
    return data_sorted

In [6]:
def save_graphs(method:str):
    path, file_list, seeds = get_files(method)
    for i in range(len(file_list)):
        data = data_sorting(path+file_list[i])
        sparsity = np.array(data["sparsity"])
        accuracy = np.array(data["accuracy"])
        plt.plot(sparsity,accuracy)
        plt.xlabel("sparsity")
        plt.ylabel("accuracy")
        plt.title(f"{method}_{seeds[i]}")
        plt.savefig(f"{path}{method}_{seeds[i]}.png")
        plt.clf()

In [12]:
def box_graphs(method:str):
    path, file_list, seeds = get_files(method)
    sorted_datas = []
    total_results = []
    for i in range(len(file_list)):
        sorted_datas.append(data_sorting(path+file_list[i]))
    results = pd.concat(sorted_datas)
    sparsitys = list(results['sparsity'].drop_duplicates())

    for i in list(sparsitys):
        total_results.append(list(results['accuracy'][results['sparsity'] == i]))
    plt.boxplot(total_results)
    plt.xticks(range(1,len(sparsitys)+1), sparsitys)
    plt.savefig(f"{path}{method}.png")
    plt.clf()

In [13]:
box_graphs('block')
box_graphs('random')
box_graphs('origin')
save_graphs('block')
save_graphs('random')
save_graphs('origin')